In [1]:
import pickle

# math and dataframes
import pandas as pd
import numpy as np

# neural network
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

# Pipeline and Evaluation
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.model_selection import cross_val_score, cross_val_predict
from imblearn.pipeline import make_pipeline

# Undersampling 
from sklearn.model_selection import StratifiedKFold
from imblearn.under_sampling import RandomUnderSampler

# jupyter notebook full-width display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# pandas formatting
pd.set_option('display.float_format', '{:.3f}'.format)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

# plotting
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
sns.set_theme()

In [2]:
df_10M = pd.read_pickle('df_10M_clustered.pickle')
X_all = pd.read_pickle('X_clustered.pickle')

In [3]:
# need to cast to float to use in tensor
X_all['mode'] = X_all['mode'].astype('float32')

In [4]:
y_column = 'in_B100'
X_columns = [
    'mode', 'acousticness', 'danceability', 'duration_ms', 'energy',
    'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence'
]
genre_columns = [
    'is_Adult_Standard', 'is_Rock', 'is_R&B', 'is_Country', 'is_Pop',
    'is_Rap', 'is_Alternative', 'is_EDM', 'is_Metal'
]
cluster_columns = ['cluster', 'cluster2']
other_columns = ['key', 'time_signature', 'genre', 'release_date']

In [5]:
# create a dict with all 'name': (X, y) key match pairs
clusters = {}

# entire predictive dataset
clusters['All'] = (X_all[X_columns+genre_columns], X_all[y_column])
# clusters['All'] = (X_all[X_columns], X_all[y_column])

# add genres
for genre in genre_columns:
    title = genre[3:]
    clusters[title] = (X_all[X_all[genre]][X_columns], X_all[X_all[genre]][y_column])
    
# add clusters
for n in sorted(X_all['cluster'].unique()):
    title = genre[3:]
    clusters['cluster1_' + str(n)] = (X_all[X_all['cluster'] == n][X_columns], X_all[X_all['cluster'] == n][y_column])
    
for n in sorted(X_all['cluster2'].unique()):
    title = genre[3:]
    clusters['cluster2_' + str(n)] = (X_all[X_all['cluster2'] == n][X_columns], X_all[X_all['cluster2'] == n][y_column])
    
# setup tuning algorithm with a small dataset
small = X_all.sample(10_000, random_state=42)
X_small = small[X_columns]
y_small = small[y_column]
clusters['small'] = (X_small, y_small)

In [6]:
# scenarios to check

metrics = [
    'balanced_accuracy', 'average_precision', 'neg_brier_score', 'f1', 'f1_micro', 
    'f1_macro', 'f1_weighted', 'neg_log_loss', 'precision', 'recall', 'roc_auc', 'jaccard'
]

cluster1_keys = [
    'cluster1_0', 'cluster1_1', 'cluster1_2', 'cluster1_3'
]

cluster2_keys = [
    'cluster2_0', 'cluster2_1', 'cluster2_2', 'cluster2_3', 'cluster2_4', 
    'cluster2_5', 'cluster2_6', 'cluster2_7', 'cluster2_8', 'cluster2_9',
]

genre_keys = [
    'Adult_Standard', 'Rock', 'R&B', 'Country', 'Pop', 'Rap', 'Alternative', 'EDM', 'Metal'
]

In [7]:
# This seems like it could work

tf_width = len(X_columns)
current_width = tf_width

how_many_hidden_layers = 8
for i in range(how_many_hidden_layers):
    if i < how_many_hidden_layers // 2 - 1:
        current_width *= 2
    else:
        current_width //= 2
    print(current_width)

22
44
88
44
22
11
5
2


In [8]:
# set up the neural network

# start with width of number of features 
# (could encode to higher dimensions)
tf_width = len(X_columns)
tf_dropout = 0.5

# setup
tf_model = Sequential(name='sequential')

# hidden layers
how_many_hidden_layers = 8
current_width = tf_width
for i in range(how_many_hidden_layers):
    if i < how_many_hidden_layers // 2 - 1:
        current_width *= 2
    else:
        current_width //= 2
    tf_model.add(Dense(current_width, activation="relu", name='dense_'+str(i)))
    tf_model.add(Dropout(tf_dropout, name='dropout_'+str(i)))

# output
tf_model.add(Dense(1, activation='sigmoid', name='output'))

# setup a checkpoint to save model
checkpoint = ModelCheckpoint('tf_model', save_best_only=True)

# compile model
tf_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC()])

In [9]:
%%time
# train the model

# pick the dataset
am_testing = False
if am_testing:
    dataset = 'small'
else:
#     dataset = 'cluster2_7' # good amount of hits, about 1M songs 
    dataset = 'Adult_Standard' # apples to apples with previous calcs
X_, y_ = clusters[dataset]

# split the dataset into train test, then separate validation set for fitting the neural network
X_train, X_test, y_train, y_test = train_test_split(X_, y_, test_size=0.2, random_state=42, stratify=y_)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42, stratify=y_train)

# undersample data
undersample = True
if undersample:
    undersampler = RandomUnderSampler(sampling_strategy='majority', random_state=42)
    X_train, y_train = undersampler.fit_resample(X_train, y_train)
    
# convert to Numpy arrays for tensorflow
X_train = np.asarray(X_train).astype('float32')
X_val = np.asarray(X_val).astype('float32')
X_test = np.asarray(X_test).astype('float32')
y_train = np.asarray(y_train).astype('float32')
y_val = np.asarray(y_val).astype('float32')
y_test = np.asarray(y_test).astype('float32')

# fit the data
tf_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=1000, callbacks=checkpoint)

Epoch 1/1000
129/129 [==============================] - 4s 24ms/step - loss: 0.7110 - auc: 0.4900 - val_loss: 0.6809 - val_auc: 0.5000
Epoch 2/1000
129/129 [==============================] - 1s 10ms/step - loss: 0.6959 - auc: 0.4997 - val_loss: 0.6818 - val_auc: 0.5000
Epoch 3/1000
129/129 [==============================] - 1s 10ms/step - loss: 0.6936 - auc: 0.5046 - val_loss: 0.6813 - val_auc: 0.5000
Epoch 4/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.6944 - auc: 0.4989 - val_loss: 0.6863 - val_auc: 0.5000
Epoch 5/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.6939 - auc: 0.5008 - val_loss: 0.6850 - val_auc: 0.5000
Epoch 6/1000
129/129 [==============================] - 1s 10ms/step - loss: 0.6936 - auc: 0.4844 - val_loss: 0.6892 - val_auc: 0.5000
Epoch 7/1000
129/129 [==============================] - 1s 10ms/step - loss: 0.6926 - auc: 0.5041 - val_loss: 0.6874 - val_auc: 0.5000
Epoch 8/1000
129/129 [==============================] - 1

Epoch 48/1000
129/129 [==============================] - 1s 10ms/step - loss: 0.6856 - auc: 0.5639 - val_loss: 0.6574 - val_auc: 0.6821
Epoch 49/1000
129/129 [==============================] - 1s 10ms/step - loss: 0.6859 - auc: 0.5522 - val_loss: 0.6585 - val_auc: 0.6894
Epoch 50/1000
129/129 [==============================] - 3s 23ms/step - loss: 0.6833 - auc: 0.5634 - val_loss: 0.6540 - val_auc: 0.6147
Epoch 51/1000
129/129 [==============================] - 1s 11ms/step - loss: 0.6855 - auc: 0.5553 - val_loss: 0.6572 - val_auc: 0.6625
Epoch 52/1000
129/129 [==============================] - 1s 11ms/step - loss: 0.6809 - auc: 0.5782 - val_loss: 0.6548 - val_auc: 0.7189
Epoch 53/1000
129/129 [==============================] - 1s 10ms/step - loss: 0.6834 - auc: 0.5717 - val_loss: 0.6576 - val_auc: 0.7278
Epoch 54/1000
129/129 [==============================] - 3s 22ms/step - loss: 0.6902 - auc: 0.5624 - val_loss: 0.6521 - val_auc: 0.6893
Epoch 55/1000
129/129 [=========================

Epoch 96/1000
129/129 [==============================] - 1s 11ms/step - loss: 0.6790 - auc: 0.5979 - val_loss: 0.6295 - val_auc: 0.7079
Epoch 97/1000
129/129 [==============================] - 1s 10ms/step - loss: 0.6786 - auc: 0.6049 - val_loss: 0.6366 - val_auc: 0.7290
Epoch 98/1000
129/129 [==============================] - 1s 10ms/step - loss: 0.6801 - auc: 0.6065 - val_loss: 0.6329 - val_auc: 0.7235
Epoch 99/1000
129/129 [==============================] - 1s 10ms/step - loss: 0.6781 - auc: 0.6148 - val_loss: 0.6373 - val_auc: 0.7283
Epoch 100/1000
129/129 [==============================] - 1s 11ms/step - loss: 0.6750 - auc: 0.6152 - val_loss: 0.6405 - val_auc: 0.7339
Epoch 101/1000
129/129 [==============================] - 1s 10ms/step - loss: 0.6768 - auc: 0.5987 - val_loss: 0.6338 - val_auc: 0.7072
Epoch 102/1000
129/129 [==============================] - 1s 11ms/step - loss: 0.6828 - auc: 0.5935 - val_loss: 0.6291 - val_auc: 0.6980
Epoch 103/1000
129/129 [=====================

Epoch 150/1000
129/129 [==============================] - 1s 10ms/step - loss: 0.6684 - auc: 0.6383 - val_loss: 0.6140 - val_auc: 0.7217
Epoch 151/1000
129/129 [==============================] - 1s 10ms/step - loss: 0.6704 - auc: 0.6256 - val_loss: 0.6183 - val_auc: 0.7287
Epoch 152/1000
129/129 [==============================] - 1s 10ms/step - loss: 0.6747 - auc: 0.6204 - val_loss: 0.6302 - val_auc: 0.7306
Epoch 153/1000
129/129 [==============================] - 1s 10ms/step - loss: 0.6659 - auc: 0.6223 - val_loss: 0.6287 - val_auc: 0.7240
Epoch 154/1000
129/129 [==============================] - 1s 10ms/step - loss: 0.6727 - auc: 0.6402 - val_loss: 0.6201 - val_auc: 0.7342
Epoch 155/1000
129/129 [==============================] - 1s 10ms/step - loss: 0.6706 - auc: 0.6206 - val_loss: 0.6213 - val_auc: 0.7418
Epoch 156/1000
129/129 [==============================] - 1s 10ms/step - loss: 0.6778 - auc: 0.6387 - val_loss: 0.6284 - val_auc: 0.7431
Epoch 157/1000
129/129 [=================

Epoch 208/1000
129/129 [==============================] - 3s 25ms/step - loss: 0.6673 - auc: 0.6528 - val_loss: 0.6050 - val_auc: 0.7412
Epoch 209/1000
129/129 [==============================] - 1s 10ms/step - loss: 0.6679 - auc: 0.6490 - val_loss: 0.6147 - val_auc: 0.7554
Epoch 210/1000
129/129 [==============================] - 1s 10ms/step - loss: 0.6625 - auc: 0.6563 - val_loss: 0.6205 - val_auc: 0.7526
Epoch 211/1000
129/129 [==============================] - 1s 10ms/step - loss: 0.6587 - auc: 0.6638 - val_loss: 0.6259 - val_auc: 0.7531
Epoch 212/1000
129/129 [==============================] - 1s 10ms/step - loss: 0.6596 - auc: 0.6582 - val_loss: 0.6210 - val_auc: 0.7436
Epoch 213/1000
129/129 [==============================] - 1s 10ms/step - loss: 0.6597 - auc: 0.6622 - val_loss: 0.6283 - val_auc: 0.7544
Epoch 214/1000
129/129 [==============================] - 1s 10ms/step - loss: 0.6653 - auc: 0.6558 - val_loss: 0.6134 - val_auc: 0.7479
Epoch 215/1000
129/129 [=================

...

Epoch 744/1000
129/129 [==============================] - 1s 10ms/step - loss: 0.6623 - auc: 0.6729 - val_loss: 0.6262 - val_auc: 0.7593
Epoch 745/1000
129/129 [==============================] - 1s 11ms/step - loss: 0.6563 - auc: 0.6626 - val_loss: 0.6257 - val_auc: 0.7345
Epoch 746/1000
129/129 [==============================] - 1s 10ms/step - loss: 0.6618 - auc: 0.6537 - val_loss: 0.6357 - val_auc: 0.7432
Epoch 747/1000
129/129 [==============================] - 1s 11ms/step - loss: 0.6613 - auc: 0.6614 - val_loss: 0.6338 - val_auc: 0.7473
Epoch 748/1000
129/129 [==============================] - 1s 11ms/step - loss: 0.6653 - auc: 0.6592 - val_loss: 0.6311 - val_auc: 0.7479
Epoch 749/1000
129/129 [==============================] - 1s 11ms/step - loss: 0.6620 - auc: 0.6515 - val_loss: 0.6330 - val_auc: 0.7318
Epoch 750/1000
129/129 [==============================] - 1s 11ms/step - loss: 0.6645 - auc: 0.6558 - val_loss: 0.6295 - val_auc: 0.7455
Epoch 751/1000
129/129 [=================

Epoch 804/1000
129/129 [==============================] - 1s 10ms/step - loss: 0.6617 - auc: 0.6475 - val_loss: 0.6405 - val_auc: 0.7405
Epoch 805/1000
129/129 [==============================] - 1s 10ms/step - loss: 0.6553 - auc: 0.6566 - val_loss: 0.6268 - val_auc: 0.7418
Epoch 806/1000
129/129 [==============================] - 1s 10ms/step - loss: 0.6676 - auc: 0.6531 - val_loss: 0.6196 - val_auc: 0.7417
Epoch 807/1000
129/129 [==============================] - 1s 10ms/step - loss: 0.6543 - auc: 0.6669 - val_loss: 0.6012 - val_auc: 0.7167
Epoch 808/1000
129/129 [==============================] - 1s 11ms/step - loss: 0.6595 - auc: 0.6622 - val_loss: 0.6516 - val_auc: 0.7449
Epoch 809/1000
129/129 [==============================] - 1s 10ms/step - loss: 0.6567 - auc: 0.6650 - val_loss: 0.6146 - val_auc: 0.7346
Epoch 810/1000
129/129 [==============================] - 1s 10ms/step - loss: 0.6561 - auc: 0.6615 - val_loss: 0.6455 - val_auc: 0.7479
Epoch 811/1000
129/129 [=================

Epoch 864/1000
129/129 [==============================] - 1s 11ms/step - loss: 0.6602 - auc: 0.6604 - val_loss: 0.6258 - val_auc: 0.7329
Epoch 865/1000
129/129 [==============================] - 1s 11ms/step - loss: 0.6581 - auc: 0.6507 - val_loss: 0.6281 - val_auc: 0.7145
Epoch 866/1000
129/129 [==============================] - 1s 11ms/step - loss: 0.6612 - auc: 0.6519 - val_loss: 0.6310 - val_auc: 0.7304
Epoch 867/1000
129/129 [==============================] - 1s 11ms/step - loss: 0.6554 - auc: 0.6593 - val_loss: 0.6385 - val_auc: 0.7342
Epoch 868/1000
129/129 [==============================] - 1s 11ms/step - loss: 0.6592 - auc: 0.6568 - val_loss: 0.6397 - val_auc: 0.7606
Epoch 869/1000
129/129 [==============================] - 1s 10ms/step - loss: 0.6625 - auc: 0.6604 - val_loss: 0.6007 - val_auc: 0.7361
Epoch 870/1000
129/129 [==============================] - 1s 11ms/step - loss: 0.6555 - auc: 0.6601 - val_loss: 0.6338 - val_auc: 0.7319
Epoch 871/1000
129/129 [=================

Epoch 924/1000
129/129 [==============================] - 1s 11ms/step - loss: 0.6583 - auc: 0.6600 - val_loss: 0.6305 - val_auc: 0.7474
Epoch 925/1000
129/129 [==============================] - 1s 11ms/step - loss: 0.6597 - auc: 0.6585 - val_loss: 0.6032 - val_auc: 0.7335
Epoch 926/1000
129/129 [==============================] - 1s 11ms/step - loss: 0.6549 - auc: 0.6693 - val_loss: 0.6123 - val_auc: 0.7338
Epoch 927/1000
129/129 [==============================] - 1s 11ms/step - loss: 0.6563 - auc: 0.6519 - val_loss: 0.6328 - val_auc: 0.7310
Epoch 928/1000
129/129 [==============================] - 1s 11ms/step - loss: 0.6533 - auc: 0.6682 - val_loss: 0.6390 - val_auc: 0.7470
Epoch 929/1000
129/129 [==============================] - 1s 11ms/step - loss: 0.6551 - auc: 0.6572 - val_loss: 0.6360 - val_auc: 0.7369
Epoch 930/1000
129/129 [==============================] - 1s 11ms/step - loss: 0.6623 - auc: 0.6536 - val_loss: 0.6191 - val_auc: 0.7407
Epoch 931/1000
129/129 [=================

Epoch 984/1000
129/129 [==============================] - 1s 11ms/step - loss: 0.6551 - auc: 0.6698 - val_loss: 0.6251 - val_auc: 0.7465
Epoch 985/1000
129/129 [==============================] - 1s 11ms/step - loss: 0.6596 - auc: 0.6719 - val_loss: 0.6085 - val_auc: 0.7373
Epoch 986/1000
129/129 [==============================] - 1s 11ms/step - loss: 0.6555 - auc: 0.6714 - val_loss: 0.6212 - val_auc: 0.7418
Epoch 987/1000
129/129 [==============================] - 1s 10ms/step - loss: 0.6563 - auc: 0.6716 - val_loss: 0.6002 - val_auc: 0.7395
Epoch 988/1000
129/129 [==============================] - 1s 11ms/step - loss: 0.6580 - auc: 0.6705 - val_loss: 0.6109 - val_auc: 0.7424
Epoch 989/1000
129/129 [==============================] - 1s 10ms/step - loss: 0.6580 - auc: 0.6724 - val_loss: 0.6109 - val_auc: 0.7512
Epoch 990/1000
129/129 [==============================] - 1s 11ms/step - loss: 0.6552 - auc: 0.6665 - val_loss: 0.6118 - val_auc: 0.7438
Epoch 991/1000
129/129 [=================

In [10]:
# predict and check results
y_pred = tf_model.predict(X_test)
print(
    '\nClassification Report\n------------------------------\n', 
    classification_report(y_test, y_pred.flatten() > 0.5)
)

1306/1306 [==============================] - 1s 653us/step

Classification Report
------------------------------
               precision    recall  f1-score   support

         0.0       0.99      0.73      0.84     41036
         1.0       0.04      0.70      0.08       736

    accuracy                           0.73     41772
   macro avg       0.52      0.71      0.46     41772
weighted avg       0.98      0.73      0.83     41772



In [11]:
# this neural network is not very certain about anything
pd.DataFrame(y_pred).describe().T

,count,mean,std,min,25%,50%,75%,max
0,41772.000,0.467,0.068,0.425,0.425,0.425,0.529,0.691


In [12]:
# 95% of predictions are in this range of certainty
pd.DataFrame(y_pred).describe([0.025, 0.975]).loc[['2.5%', '97.5%']]

,0
2.5%,0.425
97.5%,0.613


In [13]:
best_threshold = [0, 0]  # threshold, f1score
for i in range(101):
    i = i / 100
    score = f1_score(y_pred > i, y_test)
    if score > best_threshold[1]:
        best_threshold = i, score
        
best_threshold

(0.58, 0.09759650400582665)

In [16]:
# what about a higher threshold?
threshold = best_threshold[0]
print(
    '\nClassification Report\n------------------------------\n', 
    classification_report(y_test, y_pred.flatten() > threshold)
)


Classification Report
------------------------------
               precision    recall  f1-score   support

         0.0       0.99      0.89      0.94     41036
         1.0       0.06      0.36      0.10       736

    accuracy                           0.88     41772
   macro avg       0.52      0.63      0.52     41772
weighted avg       0.97      0.88      0.92     41772



In [23]:
sum(y_pred.flatten() > threshold), sum(y_test)

(4756, 736.0)

In [22]:
# predicted / actual, 36% of the actual hits were predicted
sum(y_pred.flatten() > threshold) / sum(y_test), int(sum(y_test)*0.36)

(6.461956521739131, 264)